In [ ]:
import io
import unicodedata
import re
import random
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

In [ ]:
!wget https://download.pytorch.org/tutorial/data.zip .

--2024-04-29 13:54:05--  https://download.pytorch.org/tutorial/data.zip
Resolving download.pytorch.org (download.pytorch.org)... 108.138.94.103, 108.138.94.65, 108.138.94.32, ...
Connecting to download.pytorch.org (download.pytorch.org)|108.138.94.103|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2882130 (2.7M) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>]   2.75M  --.-KB/s    in 0.04s   

2024-04-29 13:54:05 (65.8 MB/s) - ‘data.zip’ saved [2882130/2882130]

--2024-04-29 13:54:05--  http://./
Resolving . (.)... failed: No address associated with hostname.
wget: unable to resolve host address ‘.’
FINISHED --2024-04-29 13:54:05--
Total wall clock time: 0.2s
Downloaded: 1 files, 2.7M in 0.04s (65.8 MB/s)


In [ ]:
!mkdir ./datasets

In [ ]:
!unzip data.zip -d ./datasets

Archive:  data.zip
   creating: ./datasets/data/
  inflating: ./datasets/data/eng-fra.txt  
   creating: ./datasets/data/names/
  inflating: ./datasets/data/names/Arabic.txt  
  inflating: ./datasets/data/names/Chinese.txt  
  inflating: ./datasets/data/names/Czech.txt  
  inflating: ./datasets/data/names/Dutch.txt  
  inflating: ./datasets/data/names/English.txt  
  inflating: ./datasets/data/names/French.txt  
  inflating: ./datasets/data/names/German.txt  
  inflating: ./datasets/data/names/Greek.txt  
  inflating: ./datasets/data/names/Irish.txt  
  inflating: ./datasets/data/names/Italian.txt  
  inflating: ./datasets/data/names/Japanese.txt  
  inflating: ./datasets/data/names/Korean.txt  
  inflating: ./datasets/data/names/Polish.txt  
  inflating: ./datasets/data/names/Portuguese.txt  
  inflating: ./datasets/data/names/Russian.txt  
  inflating: ./datasets/data/names/Scottish.txt  
  inflating: ./datasets/data/names/Spanish.txt  
  inflating: ./datasets/data/names/Vietnamese.t

In [ ]:
SOS_token = 0
EOS_token = 1

In [ ]:
data_dir = "./datasets/data"

In [ ]:
class Lang:
  def __init__(self, name):
    self.name = name
    self.word2index = {}
    self.word2count = {}
    self.index2word = {0: "SOS", 1: "EOS"}
    self.n_words = 2 # count SOS and EOS

  def addSentence(self, sentence):
    for word in sentence.split(' '):
      self.addWord(word)

  def addWord(self, word):
    if word not in self.word2index:
      self.word2index[word] = self.n_words
      self.word2count[word] = 1
      self.index2word[self.n_words] = word
      self.n_words += 1
    else:
      self.word2count[word] += 1

In [ ]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
  return ''.join(
    c for c in unicodedata.normalize('NFD', s)
    if unicodedata.category(c) != 'Mn'
  )

In [ ]:
# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
  s = unicodeToAscii(s.lower().strip())
  s = re.sub(r"([.!?])", r" \1", s)
  s = re.sub(r"[^a-zA-Z!?]+", r" ", s)
  return s.strip()

In [ ]:
def readLangs(data_dir, lang1, lang2, reverse=False):
  print("Reading lines...")

  # Read the file and split into lines
  lines = open(f'{data_dir}/{lang1}-{lang2}.txt', encoding='utf-8').\
    read().strip().split('\n')

  # Split every line into pairs and normalize
  pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

  # Reverse pairs, make Lang instances
  if reverse:
    pairs = [list(reversed(p)) for p in pairs]
    input_lang = Lang(lang2)
    output_lang = Lang(lang1)
  else:
    input_lang = Lang(lang1)
    output_lang = Lang(lang2)

  return input_lang, output_lang, pairs

In [ ]:
MAX_LENGTH = 10

eng_prefixes = (
  "i am ", "i m ",
  "he is", "he s ",
  "she is", "she s ",
  "you are", "you re ",
  "we are", "we re ",
  "they are", "they re "
)

In [ ]:
def filterPair(p):
  return len(p[0].split(' ')) < MAX_LENGTH and \
    len(p[1].split(' ')) < MAX_LENGTH and \
    p[1].startswith(eng_prefixes)

In [ ]:
def filterPairs(pairs):
  return [pair for pair in pairs if filterPair(pair)]

Proses of preparing data:


*   Read text file and split into lines,
*   Split lines into pairs
*   Normalize text, filter by length and content
*   Make word lists from sentences in pairs





In [ ]:
def prepareData(data_dir, lang1, lang2, reverse=False):
  input_lang, output_lang, pairs = readLangs(data_dir, lang1, lang2, reverse)
  print("Read %s sentence pairs" % len(pairs))
  pairs = filterPairs(pairs)
  print("Trimmed to %s sentence pairs" % len(pairs))
  print("Counting words...")
  for pair in pairs:
      input_lang.addSentence(pair[0])
      output_lang.addSentence(pair[1])
  print("Counted words:")
  print(input_lang.name, input_lang.n_words)
  print(output_lang.name, output_lang.n_words)
  return input_lang, output_lang, pairs

In [ ]:
input_lang, output_lang, pairs = prepareData(data_dir, 'eng', 'fra', True)
print(random.choice(pairs))

Reading lines...
Read 135842 sentence pairs
Trimmed to 11445 sentence pairs
Counting words...
Counted words:
fra 4601
eng 2991
['je ne suis pas un menteur', 'i m not a liar']
